In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Masking, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, regularizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2025-09-03 15:03:10.018956: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# import training data from timeseries 

In [ ]:
def initialize_model_lstm(input_shape:tuple, dropout_rate: float = 0.3) -> Model:
    model = Sequential()
    model.add(Input(shape=(input_shape)))
    model.add(layers.Masking(mask_value=0.0))
    
    #two lstm layers to better capture 
    model.add(LSTM(units=64, activation='tanh', 
                   return_sequences=True,
                   dropout=dropout_rate,           
                   recurrent_dropout=dropout_rate))
    model.add(LSTM(units=32, activation='tanh',
                   dropout=dropout_rate,           
                   recurrent_dropout=dropout_rate))
    
    model.add(layers.Dropout(dropout_rate))
    
    #dense layer with batch normalization
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    
    #output layer with 3 classification
    model.add(layers.Dense(3, activation='softmax'))
    
    return model

In [ ]:
model_lstm = initialize_model_lstm(input_shape=X_train.shape[1:])

model_lstm.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy', 'precision', 'recall'])

TypeError: compile() missing required argument 'source' (pos 1)

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=50,
    restore_best_weights=True,
    verbose=0
)

reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1, 
        patience=3, 
        verbose=1,
        min_lr=0
    )

history = model_lstm.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    epochs=500,
    batch_size=32,
    callbacks=[es, reduce_lr],
    verbose=1
)
